In [28]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.utilities.data_transformation import load_all_files

in_dir = AbsDirPath.MERGED
df = load_all_files(in_dir)

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue.0.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue.1.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue.2.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\

In [29]:
df.shape

(1601472, 20)

In [30]:
df.head(3)

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar,s0_to_eliminate,s0_reasoning,s1_true_positive,s1_reasoning,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1,False,This content describes a function that process...,True,The content describes functionality related to...,False,Step 1: Initial Analysis\nThe content describe...,None,None
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1,True,This is a technical instruction or algorithm d...,None,None,None,None,None,None
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,2,False,This is a brief instruction about handling dow...,True,The content describes a mechanism for handling...,False,The content describes a specific implementatio...,None,None


# Add columns denoting which stage passed

In [31]:
df["prefilter_passed"] = ~df['s0_to_eliminate'].isna()
df['s0_passed'] = df["prefilter_passed"] & (df['s0_to_eliminate'] == False)
df['s1_passed'] = df['s0_passed'] & (df['s1_true_positive'] == True)
df['s2_passed'] = df['s1_passed'] & (df['s2_related_to_arch'] == True)
df['s3_passed'] = df['s2_passed'] & (~df['s3_tactic'].isna()) & (df['s3_tactic'] != "None")

In [32]:
df.columns

Index(['id', 'keyword', 'keyword_raw', 'sentence', 'qa', 'source', 'url',
       'repo_id', 'matched_word', 'unique_matched_words',
       'nunique_matched_words', 'nsimilar', 's0_to_eliminate', 's0_reasoning',
       's1_true_positive', 's1_reasoning', 's2_related_to_arch',
       's2_reasoning', 's3_tactic', 's3_response', 'prefilter_passed',
       's0_passed', 's1_passed', 's2_passed', 's3_passed'],
      dtype='object')

# Add column for tactic category

In [33]:
from cfg.tactics.tactic_to_category_map import tactic_to_category_map

df["s3_tactic_category"] = df.s3_tactic.apply(lambda x: tactic_to_category_map.get(x))
df.head()

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,...,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response,prefilter_passed,s0_passed,s1_passed,s2_passed,s3_passed,s3_tactic_category
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,False,Step 1: Initial Analysis\nThe content describe...,None,None,True,True,True,False,False,None
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,None,None,None,None,True,False,False,False,False,None
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,False,The content describes a specific implementatio...,None,None,True,True,True,False,False,None
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",...,True,The content describes a candidate generator fo...,None,The system is implementing a candidate generat...,True,True,True,True,False,None
4,allenai/scispacy/v0.5.5.code_comment.availabil...,consist,consist,A spacy pipeline component which identifies en...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,consists,"consists, Consists",...,None,None,None,None,True,False,False,False,False,None


In [34]:
df.groupby("s3_tactic_category").size()

s3_tactic_category
Control Resource Demand                       1409
Control and Observe System State              1046
Defer Binding                                   47
Detect Attacks                                  22
Detect Faults                                  136
Increase Cohesion                               14
Limit Complexity                               170
Manage Interfaces                             1524
Manage Resources                              2560
Prevent Faults                                1053
React to Attacks                                15
Recover from Attacks                             2
Recover from Faults_Preparation and Repair      66
Recover from Faults_Reintroduction               1
Reduce Coupling                                168
Reduce Size of a Module                        167
Resist Attacks                                 885
Resource Adaptation                           2345
Resource Allocation                            141
Resource Mon

# Other

In [4]:
df.source.unique()

array(['code_comment', 'docs', 'issue', 'issue_comment', 'release',
       'wiki'], dtype=object)

In [5]:
df.qa.unique()

array(['availability', 'deployability', 'energy efficiency',
       'integrability', 'interoperability', 'modifiability',
       'performance', 'reliability', 'safety', 'security', 'testability',
       'usability'], dtype=object)

In [6]:
df.repo_id.unique()

array(['allenai/scispacy/v0.5.5', 'google/deepvariant/v1.6.1',
       'OpenGene/fastp/v0.23.4', 'root-project/root/v6-32-06',
       'scverse/scanpy/1.10.2'], dtype=object)

In [7]:
df.columns

Index(['id', 'keyword', 'keyword_raw', 'sentence', 'qa', 'source', 'url',
       'repo_id', 'matched_word', 'unique_matched_words',
       'nunique_matched_words', 'nsimilar', 's0_to_eliminate', 's0_reasoning',
       's1_true_positive', 's1_reasoning', 's2_related_to_arch',
       's2_reasoning', 's3_tactic', 's3_response', 'prefilter_passed',
       's0_passed', 's1_passed', 's2_passed', 's3_passed'],
      dtype='object')

# Data funnel

In [13]:
funnel = df.groupby(['prefilter_passed',
       's0_passed', 's1_passed', 's2_passed', 's3_passed']).nsimilar.agg("sum")
funnel

prefilter_passed  s0_passed  s1_passed  s2_passed  s3_passed
False             False      False      False      False         103326
True              False      False      False      False        1220672
                  True       False      False      False          49933
                             True       False      False         525789
                                        True       False          80045
                                                   True           15085
Name: nsimilar, dtype: int64

In [14]:
total_items = df.nsimilar.sum()
funnel_df = funnel.reset_index()
funnel_df["percentage_of_total"] = funnel_df.nsimilar / total_items * 100
funnel_df

,prefilter_passed,s0_passed,s1_passed,s2_passed,s3_passed,nsimilar,percentage_of_total
0,False,False,False,False,False,103326,5.179638
1,True,False,False,False,False,1220672,61.191167
2,True,True,False,False,False,49933,2.503095
3,True,True,True,False,False,525789,26.357320
4,True,True,True,True,False,80045,4.012582
5,True,True,True,True,True,15085,0.756197


## Crosstab

In [19]:
import pandas as pd

df_tactics = df[df['s3_passed'] == True]
tactic_crosstab = pd.crosstab(df_tactics['qa'], df_tactics['s3_tactic'])
tactic_crosstab

s3_tactic,Abstract Common Services,Abstract Data Sources,Active Redundancy,Aggregate,Authenticate Actors,Authorize Actors,Bound Execution Times,Bound Queue Sizes,Cancel,Change Default Settings,...,Startup Time Binding,Static Classification,Tailor Interface,Timestamp,Transactions,Undo,Use an Intermediary,Verify Message Integrity,Vertical Scaling,Voting
qa,,,,,,,,,,,,,,,,,,,,,
availability,0,0,5,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0
deployability,10,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,5,0,0,0
energy efficiency,0,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,0,0,0,26,0
integrability,2,0,0,0,0,0,0,0,0,0,...,1,0,735,0,0,0,0,0,0,0
interoperability,1,0,0,0,0,0,0,0,0,0,...,1,0,635,0,0,0,0,0,0,0
modifiability,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,0
performance,0,0,0,0,0,0,1086,2252,0,0,...,0,0,0,0,0,0,0,0,0,0
reliability,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
safety,0,0,6,0,0,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [22]:
import pandas as pd

df_tactics = df[df['s1_passed']]
tactic_crosstab = pd.crosstab(df_tactics['source'], df_tactics['qa'], margins=True)
tactic_crosstab

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability,All
source,,,,,,,,,,,,,
code_comment,11808,18901,16894,21111,18108,20759,20169,5130,26947,21739,16788,27694,226048
docs,3167,6925,3802,5853,5717,5726,5676,1217,6101,5148,5009,9883,64224
issue,4738,12865,5184,7626,4972,7428,7036,2248,12372,6798,8507,13006,92780
issue_comment,6162,16288,5658,9728,6295,8469,9450,3313,14574,7822,11090,16540,115389
release,22,114,34,33,25,36,45,11,49,58,26,76,529
wiki,135,301,126,323,191,178,146,34,159,206,169,382,2350
All,26032,55394,31698,44674,35308,42596,42522,11953,60202,41771,41589,67581,501320


In [27]:
tactic_crosstab_totals = tactic_crosstab.loc["All"]
tactic_crosstab.div(tactic_crosstab_totals, axis=1) * 100

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability,All
source,,,,,,,,,,,,,
code_comment,45.359557,34.121024,53.296738,47.255674,51.285828,48.734623,47.431918,42.918096,44.760971,52.043284,40.366443,40.978973,45.090561
docs,12.165796,12.501354,11.994448,13.101580,16.191798,13.442577,13.348384,10.181544,10.134215,12.324340,12.044050,14.623933,12.810979
issue,18.200676,23.224537,16.354344,17.070332,14.081794,17.438257,16.546729,18.806994,20.550812,16.274449,20.454928,19.245054,18.507141
issue_comment,23.670867,29.403907,17.849707,21.775529,17.828821,19.882149,22.223790,27.716891,24.208498,18.725910,26.665705,24.474335,23.017035
release,0.084511,0.205798,0.107262,0.073868,0.070805,0.084515,0.105828,0.092027,0.081393,0.138852,0.062517,0.112458,0.105521
wiki,0.518593,0.543380,0.397501,0.723016,0.540954,0.417880,0.343352,0.284447,0.264111,0.493165,0.406357,0.565248,0.468762
All,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [13]:
df.pivot_table(index=["source", "qa"], columns=["s0_to_eliminate", "s1_true_positive", "s2_related_to_arch"], values="nsimilar", aggfunc="sum")

s0_to_eliminate                 False      
s1_true_positive                 True      
s2_related_to_arch              False True 
source       qa                            
code_comment availability       14863   798
             deployability      22537  2058
             energy efficiency  20247  1429
             integrability      25333  1935
             interoperability   21498  1535
...                               ...   ...
wiki         reliability           24    11
             safety               103    57
             security             179    44
             testability          122    54
             usability            279   110

[72 rows x 2 columns]

In [14]:
df.pivot_table(index=["source", "qa"], columns=["s0_to_eliminate"], values="nsimilar", aggfunc=["sum"], margins=True)

sum                  
s0_to_eliminate                  False     True      All
source       qa                                         
code_comment availability        17010    37627    54637
             deployability       29456    65810    95266
             energy efficiency   22643    47049    69692
             integrability       28180    68841    97021
             interoperability    24650    53301    77951
...                                ...      ...      ...
wiki         safety                167     1283     1450
             security              238     1573     1811
             testability           176     1184     1360
             usability             390     2983     3373
All                             670852  1220672  1891524

[73 rows x 3 columns]

In [19]:
df.groupby(["s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"], dropna=False).nsimilar.agg("sum")

s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic               
False            True              False               NaN                          525789
                                   True                Abstract Common Services         64
                                                       Abstract Data Sources          1153
                                                       Active Redundancy                13
                                                       Aggregate                        26
                                                                                    ...   
                                                       Vertical Scaling                 31
                                                       Voting                            3
                 False             NaN                 NaN                           49933
True             NaN               NaN                 NaN                         1220672
NaN       

In [15]:
df.groupby(["source", "s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"], dropna=False).nsimilar.agg("sum")

source        s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic               
code_comment  False            True              False               NaN                         269544
                                                 True                Abstract Common Services        38
                                                                     Abstract Data Sources          307
                                                                     Active Redundancy                3
                                                                     Aggregate                        2
                                                                                                  ...  
wiki          False            True              True                Removal from Service             1
                                                                     Separate Entities                4
                                                                     Tailo

In [16]:
df.groupby(["source", "qa", "s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"], dropna=False).nsimilar.agg("sum")

source        qa            s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic             
code_comment  availability  False            True              False               NaN                       14863
                                                               True                Active Redundancy             3
                                                                                   Condition Monitoring          6
                                                                                   Degradation                   2
                                                                                   Exception Detection           1
                                                                                                             ...  
wiki          usability     False            True              True                Maintain system models        2
                                                                                   None  

In [17]:
df.groupby(["source", "s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"]).nsimilar.agg("sum")

source        s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic               
code_comment  False            True              True                Abstract Common Services     38
                                                                     Abstract Data Sources       307
                                                                     Active Redundancy             3
                                                                     Aggregate                     2
                                                                     Authenticate Actors          11
                                                                                                ... 
wiki          False            True              True                Predictive Model              1
                                                                     Reduce Overhead               7
                                                                     Removal from Service         

In [18]:
df[df["s3_tactic"].notna()]

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar,s0_to_eliminate,s0_reasoning,s1_true_positive,s1_reasoning,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",3,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is implementing a candidate generat...
8,allenai/scispacy/v0.5.5.code_comment.deployabi...,contain,contain,in a pre-indexed approximate nearest neighbou...,deployability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,contain,"contain, Contain",2,1,False,This content describes a technical system and ...,True,The content describes how an approximate neare...,True,The content describes a system-level component...,None,The system describes an approximate nearest ne...
53,allenai/scispacy/v0.5.5.code_comment.energy ef...,current,current,A candidate generator for entity linking to a ...,energy efficiency,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,currently,"Currently, currently, CURRENTLY",3,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is being described as a candidate g...
85,allenai/scispacy/v0.5.5.code_comment.integrabi...,configur,configur,A candidate generator for entity linking to a ...,integrability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,configured,"configured, Configured",2,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is being described as a candidate g...
86,allenai/scispacy/v0.5.5.code_comment.integrabi...,sub,sub,A candidate generator for entity linking to a ...,integrability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,subject,"Subject, subject",2,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is being described as a candidate g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,scverse/scanpy/1.10.2.wiki.usability.c6c00c0ca...,help,help,API. API. Discussion . Need help? Reach out o...,usability,wiki,https://scanpy.readthedocs.io/en/stable/index....,scverse/scanpy/1.10.2,helping,helping,1,1,False,"This content is a mix of API documentation, re...",True,The content describes how users can get help t...,True,Step 1: The core topic of the content is about...,None,The system is providing information about the ...
2426,scverse/scanpy/1.10.2.wiki.usability.c9ae0abe1...,guid,guid,1.5.0 2020-05-15#. The 1.5.0 release adds a lo...,usability,wiki,https://scanpy.readthedocs.io/en/stable/releas...,scverse/scanpy/1.10.2,guide,"guide, Guide",2,1,False,This content describes a software release (ver...,True,The content describes new functionality added ...,True,The content describes updates and new function...,None,The system is being described in terms of its ...
2471,scverse/scanpy/1.10.2.wiki.usability.d81ef3b6a...,document,document,n on this while working on a PR:. Only run a s...,usability,wiki,https://scanpy.readthedocs.io/en/stable/dev/te...,scverse/scanpy/1.10.2,documentation,"documentation, Documentation, DOCUMENTATION",3,1,False,This content appears to be instructions and ad...,True,The content describes how to run tests with sp...,True,**Step 1